## Plan

- [x] FL base framework
- [x] implement non-iid sampling function following dirichlet dist

#### Attack simulation
- [x] untargeted model poisoning
    - Local model poisoning attacks to Byzantine-robust federated learning <br>
    https://github.com/vrt1shjwlkr/NDSS21-Model-Poisoning fang attack <br>
    krum-attack, trimmed-mean/median attack
- [x] targeted model poisoning
    - Analyzing federated learning through an adversarial lens <br>
    https://github.com/inspire-group/ModelPoisoning
- [x] data poisoning
    - DBA: Distributed backdoor attacks against federated learning <br>
    https://github.com/AI-secure/DBA
    
#### Defense baseline
- [x] FedAvg
- [x] Krum
- [x] Multi-Krum
- [x] Bulyan
- [x] Coordinate median
- [x] FLARE

#### Proposed method
- [x] extract PLRs
- [x] apply RBF hypersphere CKA

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Python version: 3.6


import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import copy
import time
import pickle
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
from tensorboardX import SummaryWriter

from options import args_parser
from update import LocalUpdate, test_inference, mal_inference
from models import MLP, CNNMnist, CNNFashion_Mnist, CNNCifar, Alexnet, modelC, LeNet5, GoogleNet
from utils import get_dataset, get_mal_dataset, exp_details, flatten, construct_ordered_dict
from aggregate import fedavg, multi_krum, krum, coomed, bulyan, tr_mean, fed_align, fed_cc, flare, fltrust
from attacks import get_malicious_updates_untargeted_mkrum, get_malicious_updates_untargeted_med

# python src/federated_main.py --model=cnn --dataset=cifar --gpu=0 --iid=1 --epochs=10



# Untargeted

In [3]:
class Args(object):
    
    # federated parameters (default values are set)
    epochs = 30
    num_users = 10
    frac = 1 # fraction of clients
    local_ep = 5 # num of local epoch
    local_bs = 100 # batch size
    lr = 0.001
    momentum = 0.9
    aggregation = 'mkrum' # fedavg, krum, coomed, bulyan, flare, fedcc

    # model arguments
    model = 'cnn'
    kernel_num = 9 # num of each kind of kernel
    kernel_sizes = '3,4,5' # comma-separated kernel size to use for convolution
    # num_channels = 1 # num of channels of imgs
    norm = 'batch_norm' # batch_norm, layer_norm, None
    num_filters = 32 # num of filters for conv nets -- 32 for mini-imagenet, 64 for omiglot
    max_pool = 'True' # whether use max pooling rather than strided convolutions
    
    # other arguments
    dataset = 'cifar100' # fmnist, cifar, cifar100
    
    if dataset == 'cifar100':
        num_classes = 100 
        num_channels = 3 # num of channels of imgs
    else:
        num_classes = 10
        num_channels = 1

    gpu = 0
    optimizer = 'adam'
    iid = 1 # 0 for non-iid
    alpha = 1 # noniid --> (0, 100) <-- iid
    unequal = 0 # whether to use unequal data splits for non-iid settings (0 for equal splits)
    stopping_rounds = 10 # rounds of early stopping
    verbose = 0
    seed = 1

    # malicious arguments
    mal_clients = [3] # [#attacker] 0 for no attack
    attack_type = 'untargeted_med'# untargeted_med, untargeted_mkrum
    mal_lr = 0.01

In [4]:

if __name__ == '__main__':
    start_time = time.time()

    # define paths
    path_project = os.path.abspath('..')
    logger = SummaryWriter('../logs')

    args = Args()
    exp_details(args)

    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

    for n_attacker in args.mal_clients:
        torch.cuda.empty_cache()

        # load dataset and user groups
        train_dataset, test_dataset, user_groups = get_dataset(args)

        # BUILD MODEL
        if args.model == 'cnn':
            # Convolutional neural netork
            if args.dataset == 'mnist':
                global_model = CNNMnist(args=args)
            elif args.dataset == 'fmnist':
                global_model = CNNFashion_Mnist(args=args)
            elif args.dataset == 'cifar':
                global_model = CNNCifar(args=args)
            elif args.dataset == 'cifar100':
                global_model = LeNet5(args=args)

        elif args.model == 'mlp':
            # Multi-layer preceptron
            img_size = train_dataset[0][0].shape
            len_in = 1
            for x in img_size:
                len_in *= x
                global_model = MLP(dim_in=len_in, dim_hidden=64, dim_out=args.num_classes)
        else:
            exit('Error: unrecognized model')

        # Set the model to train and send it to device.
        global_model.to(device)
        global_model.train()
        print(global_model)

        # copy weights
        global_weights = global_model.state_dict()

        # Training
        train_loss, train_accuracy = [], []
        val_acc_list, net_list = [], []
        cv_loss, cv_acc = [], []
        print_every = 1
        val_loss_pre, counter = 0, 0


        for epoch in tqdm(range(args.epochs)):
            local_weights, local_losses = [], []
            print('=========================================')
            print(f'| Global Training Round : {epoch+1} |')
            print('=========================================')

            global_model.train()
            # m = max(int(args.frac * args.num_users), 1)
            # idxs_users = np.random.choice(range(args.num_users), m, replace=False)

            flattened_local_weights = []

            
            m = max(int(args.frac * args.num_users), 1)
            idxs_users = np.random.choice(range(args.num_users), (m - args.mal_clients[0]), replace=False)

            for idx in idxs_users:
                local_model = LocalUpdate(args=args, dataset=train_dataset, idxs=user_groups[idx], logger=logger)
                w, loss = local_model.update_weights(model=copy.deepcopy(global_model), global_round=epoch)
                
                local_weights.append(copy.deepcopy(w))
                local_losses.append(copy.deepcopy(loss))
                
                # # get new model
                # new_model = copy.deepcopy(global_model)
                # new_model.load_state_dict(w)
                # acc, _ = local_model.inference(model=new_model)
                # print('user {}, loss {:.2f}, acc {:.2f}'.format(idx, loss, 100*acc))

                # flatten the local weight (list of ordereddict to a tensor of lists)
                flattened_local_weights.append(flatten(w))
            flattened_local_weights = torch.tensor(np.array(flattened_local_weights)).to(device)
            
            malicious_grads = flattened_local_weights
            
            if n_attacker > 0:
                
                # Fang attacks (untargeted MPA on mkrum and med)
                # agg_grads = torch.mean(malicious_grads, 0)
                agg_grads = torch.tensor(flatten(global_weights)).to(device)
                deviation = torch.sign(agg_grads)
                
                if args.attack_type == 'untargeted_mkrum':
                    malicious_grads = get_malicious_updates_untargeted_mkrum(malicious_grads, agg_grads, deviation, n_attacker)
                    
                elif args.attack_type == 'untargeted_med':
                    malicious_grads = get_malicious_updates_untargeted_med(malicious_grads, deviation, n_attacker)

            print(len(malicious_grads),len(flattened_local_weights), n_attacker)
            
            # update global weights
            if args.aggregation == 'fedavg':
                agg_weights = fedavg(malicious_grads)
            elif args.aggregation == 'krum':
                agg_weights, selected_idxs = krum(malicious_grads, n_attacker)
            elif args.aggregation == 'mkrum':
                agg_weights, selected_idxs = multi_krum(malicious_grads, n_attacker)
            elif args.aggregation == 'coomed':
                agg_weights = coomed(malicious_grads)
            elif args.aggregation == 'bulyan':
                agg_weights, selected_idxs = bulyan(malicious_grads, n_attacker)
            elif args.aggregation == 'trmean':
                agg_weights = tr_mean(malicious_grads, n_attacker)
            elif args.aggregation == 'fltrust':
                glob_weights = []
                glob_weights.append(flatten(global_weights))
                agg_weights = fltrust(malicious_grads, glob_weights)

            elif args.aggregation == 'flare':
                second_last_layer = list(local_weights[0].keys())[-4]
                structured_local_weights = [construct_ordered_dict(global_model, flat_weights) for flat_weights in malicious_grads]
                plrs = [(each_local[second_last_layer]) for each_local in structured_local_weights]
                agg_weights, count_dict = flare(malicious_grads, plrs)
                print(f'flare count_dict: {count_dict}')

            elif args.aggregation == 'fedcc':
                # second_last_layer = list(local_weights[0].keys())[-6]
                second_last_layer = list(local_weights[0].keys())[-4]
                glob_plr = global_weights[second_last_layer]
                # glob_plr = glob_plr.reshape((glob_plr.shape[0]*glob_plr.shape[2], glob_plr.shape[1]*glob_plr.shape[3]))
                structured_local_weights = [construct_ordered_dict(global_model, flat_weights) for flat_weights in malicious_grads]
                plrs = [(each_local[second_last_layer].reshape((glob_plr.shape[0], glob_plr.shape[1]))) for each_local in structured_local_weights]
                agg_weights, selected_idxs = fed_cc(local_weights, glob_plr, 'kernel')
                print(f'fed_cc Selected idx: {selected_idxs}')

            else:
                raise ValueError('Unknown aggregation strategy: {}'.format(args.aggregation))


            # reshape the flattened global weights into the ordereddict
            global_weights = construct_ordered_dict(global_model, agg_weights)

            # update global weights
            global_model.load_state_dict(global_weights)

            loss_avg = sum(local_losses) / len(local_losses)
            train_loss.append(loss_avg)

            # # Calculate avg training accuracy over all users at every epoch
            # list_acc, list_loss = [], []
            # global_model.eval()
            # for c in range(args.num_users):
            #     local_model = LocalUpdate(args=args, dataset=train_dataset, idxs=user_groups[c], logger=logger)
            #     acc, loss = local_model.inference(model=global_model)
            #     list_acc.append(acc)
            #     list_loss.append(loss)
            # train_accuracy.append(sum(list_acc)/len(list_acc))

            # print global training loss after every 'i' rounds
            if (epoch+1) % print_every == 0:
                print(f' \nAvg Training Stats after {epoch+1} global rounds:')
                print(f'Training Loss : {np.mean(np.array(train_loss))}')
                test_acc, test_loss = test_inference(args, global_model, test_dataset)
                
                print('%s: %s n_attacker %d fed_model val loss %.4f val acc %.4f' \
                      %(args.aggregation, args.attack_type, n_attacker, test_loss, 100*test_acc))
                print('\nGlobal model Benign Test Accuracy: {:.2f}% '.format(100*test_acc))
                
    # Test inference after completion of training
    test_acc, test_loss = test_inference(args, global_model, test_dataset)

    print(f' \n Results after {args.epochs} global rounds of training:')
    # print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
    print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

#     # Saving the objects train_loss and train_accuracy:
#     file_name = '../save/objects/{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}].pkl'.\
#         format(args.dataset, args.model, args.epochs, args.frac, args.iid, args.local_ep, args.local_bs)

#     with open(file_name, 'wb') as f:
#         pickle.dump([train_loss, train_accuracy], f)

#     print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))



Experimental details:
    Model     : cnn
    Optimizer : adam
    Learning  : 0.001
    Aggregation     : mkrum
    Global Rounds   : 30

    Federated parameters:
    IID
    Fraction of users    : 1
    Local Batch size     : 100
    Local Epochs         : 5

    Malicious parameters:
    Attackers            : [3]
    Attack Type          : untargeted_med
Files already downloaded and verified
Files already downloaded and verified
LeNet5(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=100, bias=True)
)


  0%|          | 0/30 [00:00<?, ?it/s]

| Global Training Round : 1 |


/home/hjeong_umass_edu/.conda/envs/torchgpu/lib/python3.8/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


10 7 3
 
Avg Training Stats after 1 global rounds:
Training Loss : 3.988388169663293


...

 97%|█████████▋| 29/30 [6:47:44<13:58, 838.93s/it]

mkrum: untargeted_med n_attacker 3 fed_model val loss 529.7231 val acc 16.5000

Global model Benign Test Accuracy: 16.50% 
| Global Training Round : 30 |
10 7 3
 
Avg Training Stats after 30 global rounds:
Training Loss : 0.8401157678234767


100%|██████████| 30/30 [6:58:20<00:00, 836.70s/it]

mkrum: untargeted_med n_attacker 3 fed_model val loss 540.1229 val acc 16.5200

Global model Benign Test Accuracy: 16.52% 


 
 Results after 30 global rounds of training:
|---- Test Accuracy: 16.52%


In [5]:
k

NameError: name 'k' is not defined

In [ ]:
only_weights = []
for key in global_weights:
    if 'weight' in key:
        only_weights = np.append(only_weights, global_weights[key].detach().cpu().numpy())
_, glob_svd, _, = np.linalg.svd(only_weights.reshape(1,-1))
glob_svd

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

sim_vals = []
glob_plr = glob_plr.detach().cpu()

print('euclidean')
for i in range(len(plrs)):
    val = euclidean_distances(glob_plr.reshape(1, -1), plrs[i].detach().cpu().reshape(1,-1))[0][0]
    # print(val)
    if np.isnan(val):
        sim_vals.append(0)
    else:
        sim_vals.append(val)
print(sim_vals)
    

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, paired_cosine_distances

sim_vals = []
glob_plr = glob_plr.detach().cpu()

print('cosine')
for i in range(len(plrs)):
    val = cosine_similarity(glob_plr.reshape(1, -1), plrs[i].detach().cpu().reshape(1,-1))[0][0]
    if np.isnan(val):
        sim_vals.append(0)
    else:
        sim_vals.append(val)
print(sim_vals)
    

In [ ]:
structured_local_weights = [construct_ordered_dict(global_model, flat_weights) for flat_weights in malicious_grads]

conv1_weights = [np.array(w['conv1.weight'].detach().cpu().reshape(-1)) for w in structured_local_weights]
conv1_weights = np.array(conv1_weights)
conv2_weights = [np.array(w['conv2.weight'].detach().cpu().reshape(-1)) for w in structured_local_weights]
conv2_weights = np.array(conv2_weights)
conv3_weights = [np.array(w['conv3.weight'].detach().cpu().reshape(-1)) for w in structured_local_weights]
conv3_weights = np.array(conv3_weights)
fc1_weights = [np.array(w['fc1.weight'].detach().cpu().reshape(-1)) for w in structured_local_weights]
fc1_weights = np.array(fc1_weights)
fc2_weights = [np.array(w['fc2.weight'].detach().cpu().reshape(-1)) for w in structured_local_weights]
fc2_weights = np.array(fc2_weights)

In [ ]:
num_layers = 5

_, s_layer_0, _, = np.linalg.svd(conv1_weights)
_, s_layer_1, _, = np.linalg.svd(conv2_weights)
_, s_layer_2, _, = np.linalg.svd(conv3_weights)
_, s_layer_3, _, = np.linalg.svd(fc1_weights)
_, s_layer_4, _, = np.linalg.svd(fc2_weights)


In [ ]:
from matplotlib import pyplot as plt

In [ ]:
for x, y in zip(np.arange(len(s_layer_0)), [s_layer_0, s_layer_1, s_layer_2, s_layer_3, s_layer_4]):
    print(x, y)
    plt.scatter([x], y[x], cmap='copper')

In [ ]:

for x, y in zip(np.arange(num_layers), [s_layer_0, s_layer_1, s_layer_2, s_layer_3, s_layer_4]):
    plt.scatter([x]*len(s_layer_0), y, cmap="copper")
    for i, txt in enumerate(np.arange(len(s_layer_0))):
        plt.annotate(txt, (x, y[i]))
    
plt.xticks(np.arange(num_layers))
plt.show()

In [ ]:
from sklearn.manifold import TSNE
from numpy import reshape
import seaborn as sns
import pandas as pd  

from sklearn.cluster import KMeans
from collections import Counter
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
conv1_scaled = scaler.fit_transform(conv1_weights)
conv2_scaled = scaler.fit_transform(conv2_weights)
conv3_scaled = scaler.fit_transform(conv3_weights)
fc1_scaled = scaler.fit_transform(fc1_weights)
fc2_scaled = scaler.fit_transform(fc2_weights)

tsne = TSNE(n_components=2, perplexity=5, n_iter=6000)
conv1_z = (tsne.fit_transform(conv1_weights))
conv2_z = (tsne.fit_transform(conv2_weights))
conv3_z = (tsne.fit_transform(conv3_weights))
fc1_z = (tsne.fit_transform(fc1_weights))
fc2_z = (tsne.fit_transform(fc2_weights))

In [ ]:
y=np.arange(10)
comp1 = conv1_z[:,0]
comp2 = conv1_z[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))
    
plt.show()


In [ ]:
y=np.arange(10)
comp1 = conv2_z[:,0]
comp2 = conv2_z[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))
for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))
    
plt.show()


In [ ]:
y=np.arange(10)
comp1 = conv3_z[:,0]
comp2 = conv3_z[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))
    
plt.show()


In [ ]:
y=np.arange(10)
comp1 = fc1_z[:,0]
comp2 = fc1_z[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))
for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))
    
plt.show()


In [ ]:
y=np.arange(10)
comp1 = fc2_z[:,0]
comp2 = fc2_z[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))
for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))
    
plt.show()


In [ ]:
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
conv1_scaled = scaler.fit_transform(conv1_weights)
conv2_scaled = scaler.fit_transform(conv2_weights)
conv3_scaled = scaler.fit_transform(conv3_weights)
fc1_scaled = scaler.fit_transform(fc1_weights)
fc2_scaled = scaler.fit_transform(fc2_weights)


kmeans = KMeans(n_clusters=2, random_state=0)
conv1_k = (kmeans.fit_transform(conv1_scaled))
conv2_k = (kmeans.fit_transform(conv2_scaled))
conv3_k = (kmeans.fit_transform(conv3_scaled))
fc1_k = (kmeans.fit_transform(fc1_scaled))
fc2_k = (kmeans.fit_transform(fc2_scaled))


In [ ]:
y=np.arange(10)
comp1 = conv1_k[:,0]
comp2 = conv1_k[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
y=np.arange(10)
comp1 = conv2_k[:,0]
comp2 = conv2_k[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
y=np.arange(10)
comp1 = conv3_k[:,0]
comp2 = conv3_k[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
y=np.arange(10)
comp1 = fc1_k[:,0]
comp2 = fc1_k[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
y=np.arange(10)
comp1 = fc2_k[:,0]
comp2 = fc2_k[:,1]

plt.scatter(comp1, comp2, c=y, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
conv1_scaled = scaler.fit_transform(conv1_weights)
conv2_scaled = scaler.fit_transform(conv2_weights)
conv3_scaled = scaler.fit_transform(conv3_weights)
fc1_scaled = scaler.fit_transform(fc1_weights)
fc2_scaled = scaler.fit_transform(fc2_weights)


dbscan = DBSCAN(eps=5, min_samples=1)

conv1_d = (dbscan.fit_predict(conv1_scaled))
conv2_d = (dbscan.fit_predict(conv2_scaled))
conv3_d = (dbscan.fit_predict(conv3_scaled))
fc1_d = (dbscan.fit_predict(fc1_scaled))
fc2_d = (dbscan.fit_predict(fc2_scaled))

# conv1_d = (dbscan.fit_predict(conv1_weights))
# conv2_d = (dbscan.fit_predict(conv2_weights))
# conv3_d = (dbscan.fit_predict(conv3_weights))
# fc1_d = (dbscan.fit_predict(fc1_weights))
# fc2_d = (dbscan.fit_predict(fc2_weights))


In [ ]:
y=np.arange(10)
comp1 = conv1_scaled[:,0]
comp2 = conv1_scaled[:,1]

plt.scatter(comp1, comp2, c=conv1_d, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
y=np.arange(10)
comp1 = conv2_scaled[:,0]
comp2 = conv2_scaled[:,1]

plt.scatter(comp1, comp2, c=conv2_d, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
y=np.arange(10)
comp1 = conv3_scaled[:,0]
comp2 = conv3_scaled[:,1]

plt.scatter(comp1, comp2, c=conv3_d, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
y=np.arange(10)
comp1 = fc1_scaled[:,0]
comp2 = fc1_scaled[:,1]

plt.scatter(comp1, comp2, c=fc1_d, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
y=np.arange(10)
comp1 = fc2_scaled[:,0]
comp2 = fc2_scaled[:,1]

plt.scatter(comp1, comp2, c=fc2_d, cmap=plt.cm.get_cmap("jet",10))
plt.colorbar(ticks=range(10))

for i, txt in enumerate(y):
    plt.annotate(txt, (comp1[i], comp2[i]))

plt.show()


In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

linkage_data = linkage(conv1_weights, method='single', metric='correlation')
dend = dendrogram(linkage_data)
dend['dcoord']
# plt.show()
plt.title('Fang-Med-NIID: layer1')


In [ ]:
0.027692341733213444-0.014671590015228997

In [ ]:

linkage_data = linkage(conv2_weights, method='single', metric='correlation')
dendrogram(linkage_data)['dcoord']
plt.title('Fang-Med-NIID: layer2')

# plt.show()

In [ ]:
0.1364908196119804-0.035088486275514974

In [ ]:

linkage_data = linkage(conv3_weights, method='single', metric='correlation')
dendrogram(linkage_data)['dcoord']
plt.title('Fang-Med-NIID: layer3')

# plt.show()

In [ ]:
0.3123983411700274-0.04907789487647207

In [ ]:

linkage_data = linkage(fc1_weights, method='single', metric='correlation')
dendrogram(linkage_data)['dcoord']

plt.title('Fang-Med-NIID: PLR')
# plt.show()

In [ ]:
0.4670590345070377-0.07906553072514055

In [ ]:

linkage_data = linkage(fc2_weights, method='single', metric='correlation')
dendrogram(linkage_data)['dcoord']
plt.title('Fang-Med-NIID: layer5')

# plt.show()

In [ ]:
0.08639369314601342-0.019640159432333104

In [ ]:
args.attack_type

In [ ]:
args.dataset

In [ ]:
second_last_layer

In [ ]:
exp_details(args)

###### 